In [1]:
import numpy as np
import matplotlib.pyplot as plt 
from matplotlib.ticker import (AutoMinorLocator, MultipleLocator)
import pandas as pd
from PIL import Image
import cv2

first_input_load = pd.read_csv(r'datapoint.txt')
get_input = first_input_load.to_numpy()
first_input = []
for a in get_input:
    first_input.append(float(a))
first_input = np.asarray(first_input)


def datatoimgfunc(inputlist):
    plt.rcParams["figure.figsize"] = (20, 2)
    plt.xlim([0, len(inputlist)])
    # plt.ylim([0, 120])
    plt.axis('off')
    plt.tight_layout()
    plt.plot(inputlist,'k')
    plt.savefig('datatoimage.png')
    return plt.imread('datatoimage.png')

to_use = datatoimgfunc(first_input)
plt.show()

<Figure size 2000x200 with 1 Axes>

In [ ]:
def fillblank(image_name):
    im_in = cv2.imread(image_name, cv2.IMREAD_GRAYSCALE)
    des = cv2.bitwise_not(im_in)
    contour,hier = cv2.findContours(des,cv2.RETR_CCOMP,cv2.CHAIN_APPROX_SIMPLE)

    for cnt in contour:
        cv2.drawContours(des,[cnt],0,255,-1)

    gray = cv2.bitwise_not(des)
    return gray

filled_image = fillblank('linedraw.png')
cv2.imwrite('linedraw_updated.png',filled_image)
cv2.imshow('linedraw',filled_image)
cv2.waitKey(0)

In [ ]:
img = cv2.imread('google_noisy.png', 1)

kernel = np.ones((5,5),np.float32)/25
filt_2D = cv2.filter2D(img, -1, kernel)  # Convolution using the kernel we provide
blur = cv2.blur(img, (5,5))  # Convolution with a normalized filter. Same as above for this example.
blur_gaussian = cv2.GaussianBlur(img, (5,5), 0)  # Gaussian kernel is used.
median_blur = cv2.medianBlur(img,3)
bilateral_blur = cv2.bilateralFilter(img,9,75,75)

cv2.imshow('Original', img)
cv2.imshow('2D Filter', filt_2D)
cv2.imshow('Blur', blur)
cv2.imshow('Gaussian Blur', blur_gaussian)
cv2.imshow('Median Blur', median_blur)
cv2.imshow('Bilateral Blur', bilateral_blur)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
img = cv2.imread("neuron.jpg", 0)
edges = cv2.Canny(img,100,200)   #Image, min and max values

cv2.imshow("Original Image", img)
cv2.imshow("Canny", edges)

cv2.waitKey(0)          
cv2.destroyAllWindows() 

In [17]:
img = cv2.imread("google_noisy.png", 0)

ret,th = cv2.threshold(img,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
kernel = np.ones((3,3),np.uint8)   # 3x3 kernel with all ones. 
erosion = cv2.erode(th,kernel,iterations = 1)  #Erodes pixels based on the kernel defined
dilation = cv2.dilate(erosion,kernel,iterations = 1)  #Apply dilation after erosion to see the effect. 
#Erosion followed by dilation can be a single operation called opening
opening = cv2.morphologyEx(th, cv2.MORPH_OPEN, kernel)  # Compare this image with the previous one
#Closing is opposit, dilation followed by erosion.
closing = cv2.morphologyEx(th, cv2.MORPH_CLOSE, kernel)
#Morphological gradient,difference between dilation and erosion of an image
gradient = cv2.morphologyEx(th, cv2.MORPH_GRADIENT, kernel)
#difference between input image and Opening of the image. 
tophat = cv2.morphologyEx(th, cv2.MORPH_TOPHAT, kernel)
# difference between the closing of the input image and input image.
blackhat = cv2.morphologyEx(img, cv2.MORPH_BLACKHAT, kernel)

cv2.imshow("Original Image", img)
cv2.imshow("Eroded Image", erosion)
cv2.imshow("Dilation Image", dilation)
cv2.imshow("Gradient Image", gradient)
cv2.imshow("Tophat Image", tophat)
cv2.imshow("blackhat Image", blackhat)
cv2.waitKey(0)          
cv2.destroyAllWindows() 

In [ ]:
from scipy import ndimage
from skimage import measure, color, io
from skimage.segmentation import clear_border

# Load the image
img = cv2.imread("Osteosarcoma.png")

if img is None:
    raise ValueError("Image not found. Check the file path.")
gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

ret1, thresh = cv2.threshold(gray_img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
kernel = np.ones((3, 3), np.uint8)
opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=2)
opening = clear_border(opening)
sure_bg = cv2.dilate(opening, kernel, iterations=10)

dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
ret2, sure_fg = cv2.threshold(dist_transform, 0.5 * dist_transform.max(), 255, 0)
sure_fg = np.uint8(sure_fg)
unknown = cv2.subtract(sure_bg, sure_fg)
ret3, markers = cv2.connectedComponents(sure_fg)
markers = markers + 10  # Add 10 to all labels so that sure background is not 0, but 10
markers[unknown == 255] = 0  # Mark the region of unknown with zero
markers = cv2.watershed(img, markers)
img[markers == -1] = [0, 255, 255]  # Mark boundaries with yellow color

img2 = color.label2rgb(markers, bg_label=0, bg_color=[0, 0, 0], colors=[[0, 255, 255], [255, 0, 0]])

# Display results
cv2.imshow('Original image', img)
cv2.imshow('Colored Cells', img2)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
regions = measure.regionprops(markers, intensity_image=cells)

for prop in regions:
    print('Label: {} Area: {}'.format(prop.label, prop.area))


propList = ['Area',
            'equivalent_diameter', #Added... verify if it works
            'orientation', #Added, verify if it works. Angle btwn x-axis and major axis.
            'MajorAxisLength',
            'MinorAxisLength',
            'Perimeter',
            'MinIntensity',
            'MeanIntensity',
            'MaxIntensity']    
    

output_file = open('cell_measurements.csv', 'w')
output_file.write(',' + ",".join(propList) + '\n')

for region_props in regions:
    output_file.write(str(region_props['Label']))
    for i,prop in enumerate(propList):
        if(prop == 'Area'): 
            to_print = region_props[prop]*pixels_to_um**2   #Convert pixel square to um square
        elif(prop == 'orientation'): 
            to_print = region_props[prop]*57.2958  #Convert to degrees from radians
        elif(prop.find('Intensity') < 0):          # Any prop without Intensity in its name
            to_print = region_props[prop]*pixels_to_um
        else: 
            to_print = region_props[prop]     #Reamining props, basically the ones with Intensity in its name
        output_file.write(',' + str(to_print))
    output_file.write('\n')